In [3]:
from datasets import load_dataset

dataset = load_dataset("shunk031/MSCOCO", task='semantic', streaming=True)

Resolving data files:   0%|          | 0/130 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]